In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#Silver Data Transformation

In [0]:
df = spark.read.format("delta")\
    .option("header", "True")\
        .option("inferSchema", "True")\
            .load("abfss://bronze@datalakenetflixsuresh.dfs.core.windows.net/netflix_titles")

In [0]:
df.display()

In [0]:
df = df.fillna({"duration_minutes": 0, "duration_seasons":1})

In [0]:
df.display()

In [0]:
df = df.withColumn("duration_minutes", col("duration_minutes").cast(IntegerType()))\
    .withColumn("duration_seasons", col("duration_seasons").cast(IntegerType()))

In [0]:
df.printSchema()

In [0]:
df = df.withColumn("Shortened_Title", split(col("title"),':')[0])

In [0]:
df.display()

In [0]:
#df = df.withColumn("movietype", when(col('type') == 'Movie',1).otherwise(0))
df = df.withColumn("movietype", when(col('type') == 'Movie',1)\
                                .when(col('type') == 'TV Show',0)\
                                .otherwise(2))

In [0]:
df.display(10)

In [0]:
from pyspark.sql.window import Window

In [0]:
window = Window.orderBy(col("duration_minutes").desc())
df = df.withColumn("duration_ranking", dense_rank().over(window))

In [0]:
df.display()

#Create a temporary table so that you can do some sql

In [0]:
df.createOrReplaceTempView("table1")

In [0]:
df.createOrReplaceGlobalTempView("global_table1")

In [0]:
df = spark.sql("""
               Select * from global_temp.global_table1
               """

)

In [0]:
df.display(10)

In [0]:
df = df.groupBy("type").agg(count("*").alias("TotalNumber"))
df.display()

Databricks visualization. Run in Databricks to view.

In [0]:
final_df = df

In [0]:
final_df.display()

In [0]:
final_df.write.format("delta")\
    .mode("overwrite")\
        .option("path","abfss://silver@datalakenetflixsuresh.dfs.core.windows.net/netflix_files")\
            .save()